In [1]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False


In [ ]:
df = dt.get_user_by_week_panel()
weekly_prices = dt.get_price_weekly()

c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:55: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:55: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:562: UserWarning: Converting to Period representation will drop timezone information.
  week_start = globals.data_start.to_period('W-SAT').start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:563: UserWarning: Converting to Period representation will drop timezone information.
  week_end = globals.data_end.to_period('W-SAT').start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:55: UserWarning: Converting to PeriodArray/Index

In [3]:
weekly_prices['wow_growth'] = np.log(weekly_prices['btc_price']) - np.log(weekly_prices['btc_price'].shift(1))
weekly_prices['mom_growth'] = np.log(weekly_prices['btc_price']) - np.log(weekly_prices['btc_price'].shift(4))
weekly_prices['yoy_growth'] = np.log(weekly_prices['btc_price']) - np.log(weekly_prices['btc_price'].shift(52))

In [4]:
# merge on weekly prices
df = df.merge(weekly_prices, on='week', how='left')

# remove SN employees
df = df.loc[~df['userId'].isin(globals.sn_employee_ids)].reset_index(drop=True)

# remove anon
df = df.loc[df['userId'] != globals.anon_id].reset_index(drop=True)

# output
df.to_csv(os.path.join(DATA_PATH, "user_analysis_data.csv"), index=False)

In [ ]:
res = subprocess.run([R_PATH, LOCAL_PATH + "/src/R/exit-regressions.R"], check=True, capture_output=True, text=True)
#print(res.stdout)


                               Dependent variable:        
                       -----------------------------------
                                 became_inactive          
                           (1)         (2)         (3)    
----------------------------------------------------------
unprofitable            0.900***    0.901***    0.875***  
                         (0.024)     (0.024)     (0.025)  
                                                          
pgrowth                              -0.036     -0.197**  
                                     (0.084)     (0.099)  
                                                          
unprofitable_X_pgrowth                          0.573***  
                                                 (0.185)  
                                                          
Constant                -2.499***   -2.498***   -2.492*** 
                         (0.013)     (0.014)     (0.014)  
                                                       